# Importing libraries

### Installing dependences

In [ ]:
!pip34 -q install spydal
from dal import DalContextBuilder, Inventories

In [ ]:
!pip34 install tqdm

In [ ]:
!pip34 install pandas

In [ ]:
!pip34 show pandas

In [ ]:
import scipy.stats as sts
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import json_tuple,from_json,get_json_object
from datetime import datetime, timedelta, timezone
import dateutil.tz
import datetime as dt
from pyspark.sql.functions import size
import pandas as pd
import numpy as np
from pyspark.sql.window import Window
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as f

from tqdm import tqdm
# from tqdm.notebook import tqdm 

from dal import DalContextBuilder, Inventories

import base64
from IPython.display import HTML

## Visualization in Python
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

# Defining Schemas

In [ ]:
schemaHouston = StructType((
    #- Root
    StructField("@id", StringType()),
    StructField("collectorHost", StringType()),
    StructField("deployStage", StringType()),
    StructField("deployTag", StringType()),
    #- device HARDWARE
    StructField('device', StructType([
        StructField('environmentId', StringType(), False),
    ])),
    #- experiment
    StructField("experiment", StructType((
        StructField("@id", StringType()),
        StructField("variant", StringType())       
    ))),
    #- provider
    StructField('provider', StructType([
                StructField('@id', StringType(), False),
    ])),
    StructField("published", StringType())
))

In [ ]:
schemaPulse = StructType((
    #- Root
    StructField("@type", StringType()),
    StructField("deployStage", StringType()),
    StructField("published", StringType()),
    #- device HARDWARE
    StructField("device", StructType((
        StructField("deviceType", StringType()),
        StructField("environmentId", StringType())
    ))),
    #- Object
    StructField("object", StructType((
        StructField("@id", StringType()),
        StructField("elementType", StringType()),
        StructField("@type", StringType()),
        StructField("url", StringType()),
        StructField("name",StringType())
    ))),
    #Origin
    StructField("origin", StructType([
        StructField("url", StringType())
    ])),
    #- Provider
    StructField("provider", StructType((
        StructField("@id", StringType()),
        StructField("productType", StringType())
    ))),
    #- Tracker
    StructField("tracker", StringType())
))


In [ ]:
schemaPulseRed = StructType((
    #- Root
    StructField("@type", StringType()),
    StructField("published", StringType()),
    #- device HARDWARE
    StructField("device", StructType((
        StructField("deviceType", StringType()),
        StructField("environmentId", StringType())
    ))),
    #- Object
    StructField("object", StructType((
        StructField("@id", StringType()),
        StructField("@type", StringType()),
        StructField("url", StringType()),
        StructField("category", StringType()),
        StructField("inReplyTo", StructType((
            StructField("@id", StringType()),
            StructField("category", StringType())
        )))
    ))),
    #- Provider
    StructField("provider", StructType((
        StructField("@id", StringType()),
        StructField("productType", StringType())
    ))),
    #- Actor
    StructField("actor", StructType((
        StructField("@id", StringType()),
        StructField("spt:userId", StringType())
    )))
))

# Creating Ad-hoc Filters

#### Pulse

In [ ]:
filterMsite = (col("providerProductType") == "M-Site") 

filterVitrina = (col("eventName") == "Listing Showcase Shown")
filterClickVitrina = (col("eventName") == "Listing Showcase Go To Ad")
    
filterCategory = (col("category") == "Comprar") | (col("category") == "Arrendar")

filterWeb = (col("providerProductType") == "M-Site") & (col("providerProductType") == "Web")

filterStickyClick = (col("objectId").like('%sdrn:yapocl:content:sticky:element:listing:ad_publish:%'))

filterStickyClick2 = (col('objectType') == 'UIElement') & (col('eventType')=='Click')

filterMsite = (col("providerProductType") == "M-Site")

filterAutoPromo = (col('objectUrl').like('%[native_banner]%'))

filterCreateAd = ((col('eventType_lead_1')=='Create')|
                    (col('eventType_lead_2')=='Create')|
                    (col('eventType_lead_3')=='Create')|
                  (col('eventType_lead_4')=='Create')|
                  (col('eventType_lead_5')=='Create')|
                  (col('eventType_lead_6')=='Create')|
                  (col('eventType_lead_7')=='Create')|
                    (col('eventType_lead_8')=='Create') )

filterNotCreateAd = ~((col('eventType_lead_1')=='Create')|
                    (col('eventType_lead_2')=='Create')|
                    (col('eventType_lead_3')=='Create')|
                  (col('eventType_lead_4')=='Create')|
                  (col('eventType_lead_5')=='Create')|
                  (col('eventType_lead_6')=='Create')|
                  (col('eventType_lead_7')=='Create')|
                    (col('eventType_lead_8')=='Create') )

filterMsiteForm = (col('objectUrl').like('%publica-un-aviso%')) & (~col('objectUrl').like('%m.poya%')) & (col('objectType') =='Page' )

filterPage = (col("eventType") == "View") & (col("objectType") == "Page")
    
filterListing = (col("eventType") == "View") & (col("objectType") == "Listing")

filterAdForm = (col("eventType") == "View") & (col("objectType") == "Form")

filterNewPP = (col('objectUrl').like("%&prod=%"))

filterSuccessBuy = (col('objectUrl').like("%/pvf%"))

filterSuccessFunnel = filterSuccessBuy |  ( (col('objectUrl').like("%/pagos/form%")) & (col('originUrl').like("%/dashboard%")))

filterShoppingCartFromDashboard =  ( (col('objectUrl').like("%/pagos/form%")) & (col('originUrl').like("%/dashboard%")))

filterNewBump = ( (col('objectUrl').like("%&prod=1")) | (col('objectUrl').like("%&prod=1&%")) )
filterAutoPromoSuccess = filterCreateAd & (col('originUrl').like('%[native_banner]%')) & (filterMsiteForm | filterAdForm)

#### Houston

In [ ]:
filterExperiment = (col("experiment.@id") == 'sdrn:houston')
filterDeployStage = (col("deployStage") == 'pro')

# Loading data from buckets

#### Pulse

In [ ]:
basePathPulse = "common-prod"
prefixPulse = "yellow"
sufixPulse = "sdrn:pulse"

#### Houston

In [ ]:
basePathPulse = "common-prod-houston"
prefixPulse = "yellow"
sufixPulse = "sdrn:pulse"

fromDatetimeHouston = datetime(2021, 1, 9, 0, 0, 0)
toDatetimeHouston = datetime(2021, 1, 15,0, 0, 0)

contextHouston = DalContextBuilder(spark).build()
inventoryHouston = Inventories.sparkInventory(basePathHouston, contextHouston)

In [ ]:
dfHoustonRAW = inventoryHouston.readDataFrame(prefixHouston, 
                                              [], 
                                              fromDatetimeHouston, 
                                              toDatetimeHouston,
                                              schemaHouston)

# Selecting Ad-Hoc Columns, Filtering and Creating new Features

In [ ]:
dfHouston = dfHoustonRAW\
            .filter(filterExperiment)\
            .filter(filterDeployStage)\
            .select(col('device.environmentId').alias('envIdHouston'), 
                    col('experiment.variant').alias('variant'))\
            .distinct()

In [ ]:
dfHouston.show()

In [ ]:
dfHouston.groupBy('variant')\
            .agg(countDistinct('envIdHouston')\
            .alias('users'))\
            .show()

# Transform Pulse data

In [ ]:
from pyspark.sql.functions import udf

def formVisit(x,y):
    if x is not None and y is not None:
        if ( ('publica-un-aviso' in x) and ('m.poya' not in x) and ('Page' in y)  ):
            return 'True'
        else:
            return 'False'
    else:
        return 'False'

isFormVisitUDF = udf(lambda x,y: formVisit(x,y) , StringType())


In [ ]:
def stepsList(x1,x2,x3,x4,x5,x6,x7,x8):
    return [x1,x2,x3,x4,x5,x6,x7,x8]

stepsListUDF = udf(lambda x1,x2,x3,x4,x5,x6,x7,x8 : stepsList(x1,x2,x3,x4,x5,x6,x7,x8) , StructType())

In [ ]:
condition = lambda col: ('objectType_eventType_ElementType_lead_' in col) or (col == 'date') or (col=='environmentId')

In [ ]:
# How many events forward we want to see, from ad insert visit, to create conversion funnel.
# The conversion is highly sensitive to eventsLead parameter 
#    (e.g., if we set the parameter to 1, we only will see the first event, 
#                           from ad insert visit, to create the conversion. The conversion will be near 0 percent)
basePathPulse = "common-prod"
prefixPulse = "red"
sufixPulse = "sdrn:pulse"

eventsLead = 100

# We have to set the "from date" in year, month and day variables, as int dtype.
year = 2021
month = 2
day = 28

# How many days, from start date, we want to analize. 
# We have to set daysToLoad as int greater or equal than 1.
daysToLoad = 4

# We create the dataframe where we will store the data
pdFullDF=pd.DataFrame()

dates = range(daysToLoad)
condition = lambda col: ('objectType_eventType_ElementType_lead_' in col) or (col == 'date') or (col=='environmentId')
utc_offset = lambda offset: timezone(timedelta(seconds=offset))
contextPulse = DalContextBuilder(spark).build()
inventoryPulse = Inventories.sparkInventory(basePathPulse, contextPulse)

# help(DalContextBuilder)
# help(Inventories.sparkInventory)
# help(inventoryPulse.readDataFrame)

for date in tqdm(list(dates)):
    # add utc offset: , tzinfo=utc_offset(-60*60*3) that is UTC-3, Chile's summer time
    fromDatetimePulse = datetime(year, month, day, 0, 0, 0) + timedelta(days=date) 
    toDatetimePulse = fromDatetimePulse + timedelta(days=1, hours=3) 
    filterDate = (col("date") ==  fromDatetimePulse.strftime("%Y-%m-%d") )
    # inventoryPulse.readDataFrame(prefixPulse, [sufixPulse], fromDatetimePulse, toDatetimePulse, schemaPulse)
    dfPulseRAW = inventoryPulse.readDataFrame(prefixPulse, [sufixPulse], 
                                          fromDatetimePulse, toDatetimePulse,
                                          schemaPulse)   
    dfPulse = dfPulseRAW\
            .withColumn('day',substring(col("published"),9,2))\
            .withColumn('date', substring(col("published"),0,10))\
            .withColumn('date_id', concat(substring(col("published"),0, 8),lit('01') ) )\
            .withColumn("months", split(col("date"), "-").getItem(1))\
    .select(col("published"),
            col("date"), 
            col("months"),
            col('deployStage'),
            col("@type").alias("eventType"), 
            col("object.@type").alias("objectType"), 
            col("object.@id").alias("objectId"),
            col("object.elementType").alias("elementType"),
            col("object.name").alias("objectName"),
            col("provider.productType").alias("providerProductType"),
            col('origin.url').alias('originUrl'), 
            col('object.url').alias('objectUrl'), 
            col('device.environmentId').alias('environmentId'), 
            col('day'),
            col('date_id') )\
    .na.fill({'elementType': '',
                       'objectType':'',
                         'eventType':''})\
    .withColumn("isAdForm", isFormVisitUDF(col('objectUrl'), col('objectType')))\
    .withColumn('objectType_eventType_ElementType', concat(col('objectType'),lit('_'),col('eventType'),lit('_'),col('elementType'),lit('_'),col('isAdForm') ) )\
    .drop("@type", "object", "device", "provider")

    for i in range(eventsLead):
        dfPulse = dfPulse.withColumn("objectType_eventType_ElementType_lead_{}".format(str(i+1)), lead('objectType_eventType_ElementType', i+1).over(Window.partitionBy('environmentId').orderBy('published')))
    pdFullDF = pdFullDF.append( (dfPulse.filter(filterMsite).filter(filterMsiteForm).filter(filterDate).select(*filter(condition, dfPulse.columns)).toPandas() ), ignore_index=True)
    del dfPulseRAW, dfPulse

In [ ]:
basePathPulse = "common-prod"
prefixPulse = "red"
sufixPulse = "sdrn:pulse"
schema = schemaPulseRed

In [ ]:
start_date= datetime(2021, 2, 24, 0, 0, 0)
end_date = datetime(2021, 3, 4, 0, 0, 0)


print(start_date, end_date)
context = DalContextBuilder(spark).build()
inventory = Inventories.sparkInventory(base, context)

df_base = inventory.readDataFrame(path_prefix=prefix, 
                                  provider_sdrns=[sufix], 
                                  from_datetime=start_date, 
                                  to_datetime=end_date, 
                                  schema=schema)
df_events = df_base\
    .withColumn("event_datetime", 
                from_unixtime(unix_timestamp(col("published"), "yyyy-MM-dd'T'HH:mm:ss")))\
    .withColumn('object_type', col('object.@type'))\
    .withColumn('environment_id', col('device.environmentId'))\
    .withColumn('list_id', split(col('object.inReplyTo.@id'), ':').getItem(3))\
    .withColumn("local_main_category", split(col("object.category"), " > ").getItem(0))\
    .withColumn("lead_main_category", split(col("object.inReplyTo.category"), " > ").getItem(0))\
    .withColumn("lead_category", split(col("object.inReplyTo.category"), " > ").getItem(1))\
    .withColumn("user_id", split(col("actor.spt:userId"), ":").getItem(3))\
    .withColumn('lead_type',
        when((col('@type') == 'Call') & (col('object.@type') == 'PhoneContact'), 
             "Phone_Call")\
        .when((col('@type') == 'SMS') & (col('object.@type') == 'PhoneContact'),
              "SMS_Contact" )\
        .when((col('@type') == 'Send') & (col('object.@type') == 'Message'),
              "Ad_Reply")\
        .when((col('@type') == 'Show') & (col('object.@type') == 'PhoneContact'),
              "Show_Phone")\
        .otherwise('Unknown'))\
    .select(col('event_datetime'), col('@type').alias('event_type'), 
            col('object_type'), col('lead_type'),col('environment_id'), 
            col('list_id'), col('local_main_category'), col("lead_main_category"), 
            col("lead_category"), col('user_id'))

In [ ]:
# DataFrame with leads events
df_leads = df_events\
    .withColumn('event_date', to_date(col('event_datetime')))\
    .filter(col('lead_type') != 'Unknown')\
    .filter(col('list_id').isNotNull())\
    .filter(col('list_id') != '0')


In [ ]:
pdFullDF.groupby('date').agg({'date':'count'})
mainCategoriesDict = {1000: 'main 1',
                      2000: 'main_2',
                      3000: 'main_3'}  

categories_dict={1020: 'cat 1',
                 1030: 'cat 2',
                 2030: 'cat 3'}

def createEventsList(x):
    events = list()
    start = False
    end = False
    for field in x.index:
        if 'objectType_eventType_ElementType_lead_' in field and not end:
            if x[field] == 'Page_View__True' and start is False:
                # print('PAGE VIEW TRUE FOUND')
                start = True
            if start:
                events.append(x[field])
                try:
                    if '_Create' in x[field]:
                        end = True
                except:
                    end = True
    return events
            


def listFix(x):
    oldList = x
    newList = list()
    
    keyValueFinded = None
    
    for i in oldList:
        if i is None:
            keyValueFinded = True
        elif '_True' in i:
            keyValueFinded = True
        if not keyValueFinded:
            newList.append(i)
        try:
            if '_Create' in i:
                keyValueFinded = True
        except:
             keyValueFinded = True
    return newList
            
def findCategorySelected(sortedEventsList):
    for event in sortedEventsList[::-1]:
        try:
            if 'Category Selected' in event:
                temp = []
                for char in event:
                    if char.isnumeric():
                        temp.append(char)
                return ''.join(temp)
        except:
            pass
    return 0

def findSubCategorySelected(sortedEventsList):
    for event in sortedEventsList[::-1]:
        try:
            if 'SubCategory Seleted' in event:
                temp = []
                for char in event:
                    if char.isnumeric():
                        temp.append(char)
                return ''.join(temp)
        except:
            pass
    return 0

def findLastFormEvent(sortedEventsList):
    eventsNames = ['UIElement_Image',
                    'UIElement_title',
                    'UIElement_description',
                    'UIElement_Location Region',
                    'UIElement_Seller Name',
                    'UIElement_Seller Business Name']
    for event in sortedEventsList[::-1]:
        for eventName in eventsNames:
            try:
                if eventName in event:
                    return eventName
            except:
                pass
    return 'No Form Events'

ad_insert_inmo = ['UIElement_Property Type Section',
                    'UIElement_Number of Rooms Section',
                    'UIElement_Number of Bathrooms Section',
                    'UIElement_Available Equipment Section']

In [ ]:
pdFullDF2 = pdFullDF.copy()
pdFullDF2['eventsList'] = pdFullDF2.apply((lambda x: createEventsList(x)  ),axis=1)
pdFullDF2['lastEvent'] = pdFullDF2['eventsList'].map(lambda x: findLastFormEvent(x) )

pdFullDF2['mainCategory'] = pdFullDF2['eventsList'].map(lambda x:findCategorySelected(x) )
    
pdFullDF2['mainCategoryName'] = pdFullDF2['mainCategory'].map(lambda x: mainCategoriesDict[int(x)] )

pdFullDF2['subCategory'] = pdFullDF2['eventsList'].map(lambda x:findSubCategorySelected(x) )

pdFullDF2['categoryName'] = pdFullDF2['subCategory'].map(lambda x: categories_dict[int(x)] )

pdFullDF2['adInserted'] = pdFullDF2['eventsList'].map(lambda x: 'ClassifiedAd_Create__False' in x ) 
    
print(pdFullDF2.groupby(['date','adInserted']).agg({'adInserted':'count'}).unstack() )

In [ ]:
def check_category_integrity(pdFullDF2, ad_insert_inmo):
    new_dict = {'date':'', 'environmentId':'', 'mainCategory': '', 'subCategory': '', 'eventsList': [], 'lastEvent': '', 
               'adInserted': False}
    dfAnalysis = pd.DataFrame(columns=new_dict.keys())
    new_dict = {'date':'', 'environmentId':'', 'mainCategory': '', 'subCategory': '', 'eventsList': [], 'lastEvent': '', 
               'adInserted': False}
    dfFailed = pd.DataFrame(columns=new_dict.keys())
    new_dict = {'date':'', 'environmentId':'', 'mainCategory': '', 'subCategory': '', 'eventsList': [], 'lastEvent': '', 
               'adInserted': False}
    dfInserted = pd.DataFrame(columns=new_dict.keys())
    
    funnel_ok = 0
    funnel_main = 0
    funnel_sub = 0
    funnel_complete = 0
    funnel_fail = 0
    
    average_events = 0
    number_events = 0
    
    for i, row in pdFullDF2.iterrows():
        maincat = row['mainCategory']
        subcat = row['subCategory']
        eventss = row['eventsList']
        # print(eventss)
        if len(eventss) > 0:
            average_events += len(eventss[1:])
            number_events += 1
        go = False
        for eventt in range(len(eventss)):
            eventtt = eventss[eventt]
            for eve in ad_insert_inmo:
                if eventtt is not None:
                    eve1 = eve
                    eventtt1 = eventtt
                    if '_False' in eventtt:
                        eventtt1 = eventtt[0:len(eventtt)-len("_False")]
                    elif '_True' in eventtt:
                        eventtt1 = eventtt[0:len(eventtt)-len("_True")]
                    if '_False' in eve:
                        eve1 = eve[0:len(eve)-len("_False")]
                    elif '_True' in eve:
                        eve1 = eve[0:len(eve)-len("_True")]
                    if eve1 in eventtt1:
                        go = True
                        break
        if go:
            if len(eventss) > 1:
                col_dict_2 = {'date':row['date'], 'environmentId':row['environmentId'], 
                              'mainCategory': row['mainCategory'], 'subCategory': row['subCategory'], 
                              'eventsList': row['eventsList'][1:], 'lastEvent': row['lastEvent'],
                             'adInserted': row['adInserted']}
                dfAnalysis = dfAnalysis.append(col_dict_2, ignore_index=True)
                if row['adInserted']:
                    dfInserted = dfInserted.append(col_dict_2, ignore_index=True)
                else:
                    dfFailed = dfFailed.append(col_dict_2, ignore_index=True)
            
            if maincat == 0 and subcat == 0:
                funnel_fail += 1
            else:
                funnel_ok += 1
                if maincat != 0 and subcat != 0:
                    funnel_complete +=1
                elif maincat == 0 and subcat != 0:
                    funnel_sub += 1
                elif maincat != 0 and subcat == 0:
                    funnel_main += 1
    if number_events > 0:
        print("Average number of events per valid funnel: {}".format(str(average_events/number_events)))
        print("Funnel success rate: {} percent".format(str(100*funnel_ok/(funnel_ok + funnel_fail))))
        print("Funnel fail rate: {} percent\n".format(str(100*funnel_fail/(funnel_ok + funnel_fail))))
        print("Category and Subcategory complete rate: {} percent".format(str(100*funnel_complete/funnel_ok)))
        print("Category complete rate: {} percent".format(str(100*funnel_main/funnel_ok)))
        print("Subcategory complete rate: {} percent".format(str(100*funnel_sub/funnel_ok)))
    
    return dfAnalysis, dfFailed, dfInserted

In [ ]:
def distance_to_fail(x, points):
    fail_dict = {}
    for ele in x:
        okk = False
        for eve in points:
            if ele is not None:
                ele1 = ele
                eve1 = eve
                if '_False' in ele:
                    ele1 = ele[0:len(ele)-len("_False")]
                elif '_True' in ele:
                    ele1 = ele[0:len(ele)-len("_True")]
                if '_False' in eve:
                    eve1 = eve[0:len(eve)-len("_False")]
                elif '_True' in eve:
                    eve1 = eve[0:len(eve)-len("_True")]
                if eve1 in ele1:
                    okk = True
        if not okk and ele is not None:
            ind1 = x.index(ele)
            if ele not in fail_dict.keys():
                ele2 = ele
                if '_False' in ele:
                    ele2 = ele[0:len(ele)-len("_False")]
                elif '_True' in ele:
                    ele2 = ele[0:len(ele)-len("_True")]
                fail_dict[ele2] = 0
            for elem in x:
                if elem is not None:
                    ok = False
                    for eve in points:
                        elem1 = elem
                        eve1 = eve
                        if '_False' in elem:
                            elem1 = elem[0:len(elem)-len("_False")]
                        elif '_True' in elem:
                            elem1 = elem[0:len(elem)-len("_True")]
                        if '_False' in eve:
                            eve1 = eve[0:len(eve)-len("_False")]
                        elif '_True' in eve:
                            eve1 = eve[0:len(eve)-len("_True")]
                        if eve1 in elem1:
                            ok = True
                    if ok: 
                        ind2 = x.index(elem)
                        diff = ((ind1 - ind2)**2)**0.5
                        ele2 = ele
                        if '_False' in ele:
                            ele2 = ele[0:len(ele)-len("_False")]
                        elif '_True' in ele:
                            ele2 = ele[0:len(ele)-len("_True")]
                        fail_dict[ele2] = fail_dict[ele2] + diff
    return fail_dict

In [ ]:
lastin = 0
plt.rcParams['figure.figsize'] = (15,10)
def full_analysis(dfAnalysis, dfFailed, dfInserted, op=0):
    global lastin
    if op == 0:
        dfAnalysis.to_csv("NAA_IMPACT_ALL_FIELDS.csv")
        rows_examine = len(dfAnalysis['eventsList'])
        events_dict = {}
        keys = []
        values = []
        agg = 0
        for ind in range(rows_examine):
            event_list = dfAnalysis['eventsList'][ind]
            let = False
            for event in event_list:
                for eve in ad_insert_inmo:
                    if event is not None:
                        eve1 = eve
                        event1 = event
                        if '_False' in event:
                            event1 = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event1 = event[0:len(event)-len("_True")]
                        if '_False' in eve:
                            eve1 = eve[0:len(eve)-len("_False")]
                        elif '_True' in eve:
                            eve1 = eve[0:len(eve)-len("_True")]
                        if eve1 in event1:
                            let = True
                            break
            if let:
                for event in event_list:
                    if event is not None:
                        if '_False' in event:
                            event = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event = event[0:len(event)-len("_True")]
                        if event not in events_dict.keys():
                            events_dict[event] = 1
                        else:
                            events_dict[event] = events_dict[event] + 1
        tags_resume = pd.DataFrame.from_dict(events_dict, orient='index')
        tags_resume.to_csv("NAA_TAGS_ADINSERT.csv")

        for i in events_dict.items():
            agg += i[1]
        for i in events_dict.items():
            keys.append(i[0])
            values.append(i[1]*(1/agg))
        general = pd.DataFrame({"Tags": keys, 
                               "Frequency %": values}) 
        general = general.sort_values('Frequency %', ascending=False)
        general = general.iloc[0:25,:]
        general['Frequency %'] = general['Frequency %'].apply(lambda x: x*100)
        ax0 = sns.barplot(x='Tags', 
                    y="Frequency %", data=general,  
                    order=general.sort_values('Frequency %',ascending = False).Tags)
        ax0.set_title('Normalized ad insert general tag frequency %')
        ax0.set_xticklabels(ax0.get_xticklabels(),rotation=90)
        print(general)
        return general
    # ------------------------------------------------------------------------------
    elif op == 1:
        convDF = dfAnalysis.groupby(['date','adInserted']).agg({'adInserted':'count'}).unstack().reset_index()
        convDF.columns = ['date','false','true']
        convDF['conv'] = convDF.apply((lambda x: x['true']/(x['true']+x['false']) ), axis=1)
        print(convDF['true'].mean())
        print(convDF['false'].mean())
        print(convDF['conv'].mean())
    # ------------------------------------------------------------------------------
    elif op == 2:
        last_tag = {}
        keys = []
        values = []
        
        average_events = 0
        number_events = 0
        agg = 0
        for i, row in dfFailed.iterrows():
            let = False
            for event in row['eventsList']:
                for eve in ad_insert_inmo:
                    if event is not None:
                        eve1 = eve
                        event1 = event
                        if '_False' in event:
                            event1 = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event1 = event[0:len(event)-len("_True")]
                        if '_False' in eve:
                            eve1 = eve[0:len(eve)-len("_False")]
                        elif '_True' in eve:
                            eve1 = eve[0:len(eve)-len("_True")]
                        if eve1 in event1:
                            let = True
                            break
            if let:
                tag = row['lastEvent']
                if tag is not None:
                    if '_False' in tag:
                        tag = tag[0:len(tag)-len("_False")]
                    elif '_True' in tag:
                        tag = tag[0:len(tag)-len("_True")]
                    if row['lastEvent'] not in last_tag.keys():
                        last_tag[row['lastEvent']] = 1
                    else:
                        last_tag[row['lastEvent']] = last_tag[row['lastEvent']] + 1
                    if len(row['eventsList']) > 0:
                        average_events += len(row['eventsList'])
                        number_events += 1
        for i in last_tag.items():
            agg += i[1]
        for i in last_tag.items():
            keys.append(i[0])
            values.append(i[1]*(1/agg))
        lastin = pd.DataFrame({"Tags": keys, 
                               "Frequency %": values}) 
        lastin = lastin.sort_values('Frequency %', ascending=False)
        lastin = lastin.iloc[0:25,:]
        lastin['Frequency %'] = lastin['Frequency %'].apply(lambda x: x*100)

        ax = sns.barplot(x='Tags', 
                    y="Frequency %", data=lastin,  
                    order=lastin.sort_values('Frequency %',ascending = False).Tags)
        ax.set_title('Normalized last Tag before insert fail')
        ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
        print("Normalized average number of events per valid funnel: {}".format(str(average_events/number_events)))
        print(lastin)
        return lastin
    # ------------------------------------------------------------------------------
    elif op == 3:
        rows_examine = len(dfFailed['eventsList'])
        events_dict = {}
        keys = []
        values = []
        agg = 0
        for ind in range(rows_examine):
            event_list = dfFailed['eventsList'][ind]
            let = False
            for event in event_list:
                for eve in ad_insert_inmo:
                    if event is not None:
                        eve1 = eve
                        event1 = event
                        if '_False' in event:
                            event1 = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event1 = event[0:len(event)-len("_True")]
                        if '_False' in eve:
                            eve1 = eve[0:len(eve)-len("_False")]
                        elif '_True' in eve:
                            eve1 = eve[0:len(eve)-len("_True")]
                        if eve1 in event1:
                            let = True
                            break
            if let:
                for event in event_list:
                    if event is not None:
                        if '_False' in event:
                            event = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event = event[0:len(event)-len("_True")]
                        if "Page_View" not in event:
                            if event not in events_dict.keys():
                                events_dict[event] = 1
                            else:
                                events_dict[event] = events_dict[event] + 1
        for i in events_dict.items():
            agg += i[1]
        for i in events_dict.items():
            keys.append(i[0])
            values.append(i[1]*(1/agg))
        fail = pd.DataFrame({"Tags": keys, 
                               "Frequency %": values}) 
        fail = fail.sort_values('Frequency %', ascending=False)
        fail = fail.iloc[0:25,:]
        fail['Frequency %'] = fail['Frequency %'].apply(lambda x: x*100)
        ax1 = sns.barplot(x='Tags', 
                    y="Frequency %", data=fail,  
                    order=fail.sort_values('Frequency %',ascending = False).Tags)
        ax1.set_title('Normalized ad insert tag frequency % of insert fail')
        ax1.set_xticklabels(ax1.get_xticklabels(),rotation=90)
        print(fail)
        return fail
    # ------------------------------------------------------------------------------
    elif op == 4:
        rows_examine = len(dfInserted['eventsList'])
        events_dict = {}
        keys = []
        values = []
        average_events = 0
        number_events = 0
        agg = 0
        for ind in range(rows_examine):
            event_list = dfInserted['eventsList'][ind]
            let = False
            for event in event_list:
                for eve in ad_insert_inmo:
                    if event is not None:
                        eve1 = eve
                        event1 = event
                        if '_False' in event:
                            event1 = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event1 = event[0:len(event)-len("_True")]
                        if '_False' in eve:
                            eve1 = eve[0:len(eve)-len("_False")]
                        elif '_True' in eve:
                            eve1 = eve[0:len(eve)-len("_True")]
                        if eve1 in event1:
                            let = True
                            break
            if let:
                if len(event_list) > 0:
                    average_events += len(event_list)
                    number_events += 1
                for event in event_list:
                    if event is not None:
                        if '_False' in event:
                            event = event[0:len(event)-len("_False")]
                        elif '_True' in event:
                            event = event[0:len(event)-len("_True")]
                        if "Page_View" not in event:
                            if event not in events_dict.keys():
                                events_dict[event] = 1
                            else:
                                events_dict[event] = events_dict[event] + 1
        for i in events_dict.items():
            agg += i[1]
        for i in events_dict.items():
            keys.append(i[0])
            values.append(i[1]*(1/agg))
        success = pd.DataFrame({"Tags": keys, 
                               "Frequency %": values}) 
        success = success.sort_values('Frequency %', ascending=False)
        success = success.iloc[0:25,:]
        success['Frequency %'] = success['Frequency %'].apply(lambda x: x*100)
        ax2 = sns.barplot(x='Tags', 
                    y="Frequency %", data=success,  
                    order=success.sort_values('Frequency %',ascending = False).Tags)
        ax2.set_title('Normalized ad insert tag frequency % of insert success')
        ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90)
        print("Average number of events per valid funnel: {}".format(str(average_events/number_events)))
        print(success)
        return success
    # ------------------------------------------------------------------------------
    elif op == 5:
        failure_points = lastin['Tags'].tolist()
        dfFailed['chronicles'] = dfFailed['eventsList'].map(lambda x: distance_to_fail(x, failure_points) )
        event_distances = {}
        keys = []
        values = []
        for i, row in dfFailed.iterrows():
            events = row['chronicles']
            for elem in events.keys():
                if elem not in event_distances.keys():
                    event_distances[elem] = events[elem]
                else:
                    event_distances[elem] = event_distances[elem] + events[elem]

        for i in event_distances.items():
            if i[0] is not None:
                keys.append(i[0])
                values.append(i[1])
        distances = pd.DataFrame({"Tags": keys, 
                               "Distance": values}) 
        distances = distances.sort_values('Distance', ascending=True)
        distances = distances.iloc[0:25,:]
        ax3 = sns.barplot(x='Tags', 
                    y="Distance", data=distances,  
                    order=distances.sort_values('Distance',ascending = True).Tags)
        ax3.set_title('Ad insert tag distances to journey breaks')
        ax3.set_xticklabels(ax3.get_xticklabels(),rotation=90)
        for f in failure_points:
            print(f)
        print(distances)
        return distances

In [ ]:
dfHouston2 = dfHouston.toPandas()
print(pdFullDF2.environmentId.astype('str'))
print(dfHouston2.envIdHouston.astype('str'))

In [ ]:
pdFullDF2.head()

In [ ]:
dfHouston2.head()

In [ ]:
all_people = df_leads.toPandas().copy().merge(pdFullDF2, left_on='environment_id', right_on='environmentId')

In [ ]:
all_people.head()

In [ ]:
all_cases = dfHouston2.copy().merge(pdFullDF2, left_on='envIdHouston', right_on='environmentId')
all_cases.head()

In [ ]:
all_cases_2 = all_cases.drop_duplicates(subset=all_cases.copy().drop('eventsList', axis=1).columns.values, keep='last')

In [ ]:
print(len(all_cases))
print(len(all_cases_2))
all_cases_2.head()

In [ ]:
dfAnalysis_A, dfFailed_A, dfInserted_A = check_category_integrity(all_cases_2.loc[all_cases_2['variant'] == 'A'], ad_insert_inmo)
print("Valid funnels in all A funnels: {} percent\n".format(str(100*len(dfAnalysis_A)/len(all_cases_2))) )
dfAnalysis_B, dfFailed_B, dfInserted_B = check_category_integrity(all_cases_2.loc[all_cases_2['variant'] == 'B'], ad_insert_inmo)
print("Valid funnels in all B funnels: {} percent\n".format(str(100*len(dfAnalysis_B)/len(all_cases_2))) )

In [ ]:
print(len(dfAnalysis_A), len(dfAnalysis_B))
dfAnalysis_B = dfAnalysis_B.iloc[0:len(dfAnalysis_A),:]

# Analysis of variant A

In [ ]:
op = 1
print('ANALYSIS FAILED A FUNNELS OP {}'.format(str(op)))
full_analysis(dfAnalysis_A, dfFailed_A, dfInserted_A, op)

In [ ]:
op = 2
print('ANALYSIS FAILED A FUNNELS OP {}'.format(str(op)))
last_A = full_analysis(dfAnalysis_A, dfFailed_A, dfInserted_A, op)

In [ ]:
op = 3
print('ANALYSIS FAILED A FUNNELS OP {}'.format(str(op)))
fail_A = full_analysis(dfAnalysis_A, dfFailed_A, dfInserted_A, op)

In [ ]:
op = 5
print('ANALYSIS FAILED A FUNNELS OP {}'.format(str(op)))
chronicle_A = full_analysis(dfAnalysis_A, dfFailed_A, dfInserted_A, op)

In [ ]:
op = 4
print('ANALYSIS SUCCESFUL A FUNNELS OP {}'.format(str(op)))
success_A = full_analysis(dfAnalysis_A, dfFailed_A, dfInserted_A, op)

# Analysis of variant B

In [ ]:
op = 1
print('ANALYSIS FAILED B FUNNELS OP {}'.format(str(op)))
full_analysis(dfAnalysis_B, dfFailed_B, dfInserted_B, op)

In [ ]:
op = 2
print('ANALYSIS FAILED B FUNNELS OP {}'.format(str(op)))
last_B = full_analysis(dfAnalysis_B, dfFailed_B, dfInserted_B, op)

In [ ]:
op = 3
print('ANALYSIS FAILED B FUNNELS OP {}'.format(str(op)))
fail_B = full_analysis(dfAnalysis_B, dfFailed_B, dfInserted_B, op)

In [ ]:
op = 5
print('ANALYSIS FAILED B FUNNELS OP {}'.format(str(op)))
chronicle_B = full_analysis(dfAnalysis_B, dfFailed_B, dfInserted_B, op)

In [ ]:
op = 4
print('ANALYSIS SUCCESFUL B FUNNELS OP {}'.format(str(op)))
success_B = full_analysis(dfAnalysis_B, dfFailed_B, dfInserted_B, op)

# Analyze differences between variants

In [ ]:
last_A.rename(columns = {'Frequency %':'Frequency % A'}, inplace = True) 
fail_A.rename(columns = {'Frequency %':'Frequency % A'}, inplace = True) 
success_A.rename(columns = {'Frequency %':'Frequency % A'}, inplace = True) 
chronicle_A.rename(columns = {'Distance':'Distance A'}, inplace = True) 

last_B.rename(columns = {'Frequency %':'Frequency % B'}, inplace = True) 
fail_B.rename(columns = {'Frequency %':'Frequency % B'}, inplace = True) 
success_B.rename(columns = {'Frequency %':'Frequency % B'}, inplace = True) 
chronicle_B.rename(columns = {'Distance':'Distance B'}, inplace = True) 

last_AB = last_A.merge(last_B, how='inner', on='Tags')
fail_AB = fail_A.merge(fail_B, how='inner', on='Tags')
success_AB = success_A.merge(success_B, how='inner', on='Tags')
chronicle_AB = chronicle_A.merge(chronicle_B, how='inner', on='Tags’)

last_A = last_A.drop('Frequency % A', axis=1)
last_B = last_B.drop('Frequency % B', axis=1)
fail_A = fail_A.drop('Frequency % A', axis=1)
fail_B = fail_B.drop('Frequency % B', axis=1)
success_A = success_A.drop('Frequency % A', axis=1)
success_B = success_B.drop('Frequency % B', axis=1)
chronicle_A = chronicle_A.drop('Distance A', axis=1)
chronicle_B = chronicle_B.drop('Distance B', axis=1)

last_not_B = last_A[~last_A.isin(last_B)].dropna()
last_not_A = last_B[~last_B.isin(last_A)].dropna()
fail_not_B = fail_A[~fail_A.isin(fail_B)].dropna()
fail_not_A = fail_B[~fail_B.isin(fail_A)].dropna()
success_not_B = success_A[~success_A.isin(success_B)].dropna()
success_not_A = success_B[~success_B.isin(success_A)].dropna()
chronicle_not_B = chronicle_A[~chronicle_A.isin(chronicle_B)].dropna()
chronicle_not_A = chronicle_B[~chronicle_B.isin(chronicle_A)].dropna()

In [ ]:
with pd.ExcelWriter('ad_insert_AB_testing_Houston_filtered.xlsx') as writer:  
    last_AB.to_excel(writer, sheet_name='comparison last tag')
    fail_AB.to_excel(writer, sheet_name='comparison fail funnel')
    success_AB.to_excel(writer, sheet_name='comparison success funnel')
    chronicle_AB.to_excel(writer, sheet_name='comparison chronicle')
    last_not_B.to_excel(writer, sheet_name='last tag A not B')
    last_not_A.to_excel(writer, sheet_name='last tag B not A')
    fail_not_B.to_excel(writer, sheet_name='fail funnel A not B')
    fail_not_A.to_excel(writer, sheet_name='fail funnel B not A')
    success_not_B.to_excel(writer, sheet_name='success funnel A not B')
    success_not_A.to_excel(writer, sheet_name='success funnel B not A')
    chronicle_not_B.to_excel(writer, sheet_name='chronicle A not B')
    chronicle_not_A.to_excel(writer, sheet_name='chronicle B not A')